# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_TD,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,castro,-24.7911,-50.0119,67.01,96,100,6.69,BR,1644000054
1,1,margate,51.3813,1.3862,42.89,70,1,4.00,GB,1644000056
2,2,rikitea,-23.1203,-134.9692,78.46,71,9,10.63,PF,1644000000
3,3,hasaki,35.7333,140.8333,28.69,44,97,21.79,JP,1644000061
4,4,jamestown,42.0970,-79.2353,21.09,62,100,3.00,US,1643999782


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
gmaps.configure(api_key=g_key)

In [16]:
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
ideal_weather_df = city_data_df.loc[
    (city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 80) & \
    (city_data_df["Wind Speed"] < 10) & \
    (city_data_df["Cloudiness"] == 0)
].dropna()

print(len(ideal_weather_df))

ideal_weather_df.head()

10


,City_TD,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,cape town,-33.9258,18.4232,70.21,63,0,8.01,ZA,1644000013
102,102,geraldton,-28.7667,114.6000,72.01,53,0,3.44,AU,1644000201
118,118,perth,-31.9333,115.8333,74.66,48,0,0.00,AU,1644000088
157,157,makakilo city,21.3469,-158.0858,74.19,79,0,4.61,US,1644000444
239,239,kahului,20.8947,-156.4700,75.85,70,0,4.61,US,1644000526


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

<ipython-input-18-34baa34779ac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Hotel Name
7,cape town,ZA,-33.9258,18.4232,
102,geraldton,AU,-28.7667,114.6000,
118,perth,AU,-31.9333,115.8333,
157,makakilo city,US,21.3469,-158.0858,
239,kahului,US,20.8947,-156.4700,
309,sao filipe,CV,14.8961,-24.4956,
321,cabo san lucas,MX,22.8909,-109.9124,
404,wamba,NG,8.9333,8.6000,
440,isla vista,US,34.4133,-119.8610,
537,riyadh,SA,24.6877,46.7219,


In [20]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url ="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params).json()
    
        
    try:
        hotel_df.loc[index,"Hotel Name"] = name_address["results"][0]["name"]
    
    except:
        print(f"Error finding lodging in {row['City']}. Skipping...")
        pass

    
hotel_df
    

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,City,Country,Lat,Lng,Hotel Name
7,cape town,ZA,-33.9258,18.4232,Southern Sun Waterfront Cape Town
102,geraldton,AU,-28.7667,114.6000,Broadwater Mariner Resort
118,perth,AU,-31.9333,115.8333,Beatty Lodge
157,makakilo city,US,21.3469,-158.0858,Marriott's Ko Olina Beach Club
239,kahului,US,20.8947,-156.4700,Maui Seaside Hotel
309,sao filipe,CV,14.8961,-24.4956,Hotel Xaguate
321,cabo san lucas,MX,22.8909,-109.9124,Hotel Tesoro Los Cabos
404,wamba,NG,8.9333,8.6000,Roma Zee Guest House Annex
440,isla vista,US,34.4133,-119.8610,Best Western Plus South Coast Inn
537,riyadh,SA,24.6877,46.7219,"InterContinental Riyadh, an IHG Hotel"


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))